<a href="https://colab.research.google.com/github/louistrue/learn-ifc-bfh25-D/blob/main/BFH-25-PropertySets-Export.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 🧱 IFC PropertySets Export to CSV

Extrahiert alle PropertySets aus einem IFC-Modell und speichert sie als CSV.

**Outputs:**
- 📄 `property_sets_detailed.csv` - Alle Properties inkl. Element-Details
- 📄 `property_sets_summary.csv` - PropertySet-Property-Kombinationen

In [ ]:
%pip install ifcopenshell


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import ifcopenshell
import ifcopenshell.util.element as uel
import pandas as pd
from collections import defaultdict


In [ ]:
# 📂 IFC-Modell laden
# Für Colab: Datei hochladen
# Für lokal: Pfad anpassen
try:
    from google.colab import files
    print("🔼 Google Colab erkannt - Bitte IFC-Datei hochladen...")
    uploaded = files.upload()
    IFC_FILE = list(uploaded.keys())[0]
    print(f"✅ Datei hochgeladen: {IFC_FILE}")
except:
    # Lokaler Pfad
    IFC_FILE = "Modelle/BFH-25/02_BIMcollab_Example_ARC.ifc"
    print(f"💻 Lokaler Modus - Lade: {IFC_FILE}")

model = ifcopenshell.open(IFC_FILE)
print(f"✅ Modell geladen: {model.schema} | {len(model.by_type('IfcElement'))} Elemente")


💻 Lokaler Modus - Lade: Modelle/BFH-25/02_BIMcollab_Example_ARC.ifc


✅ Modell geladen: IFC2X3 | 1354 Elemente


In [ ]:
# 📊 PropertySets extrahieren
detailed_data = []
summary_data = defaultdict(set)

print("🔍 Extrahiere PropertySets...")

for element in model.by_type('IfcElement'):
    element_id = element.GlobalId
    element_type = element.is_a()
    element_name = getattr(element, 'Name', None) or 'Unnamed'
    
    # PropertySets abrufen
    psets = uel.get_psets(element)
    
    for pset_name, properties in psets.items():
        if not isinstance(properties, dict):
            continue
        
        for prop_name, prop_value in properties.items():
            if prop_name == 'id':
                continue
            
            # Detaillierte Daten
            detailed_data.append({
                'Element_GlobalId': element_id,
                'Element_Class': element_type,
                'Element_Name': element_name,
                'PropertySet': pset_name,
                'Property': prop_name,
                'Value': str(prop_value)
            })
            
            # Summary
            summary_data[pset_name].add(prop_name)

print(f"✅ {len(detailed_data)} Property-Werte extrahiert")
print(f"✅ {len(summary_data)} PropertySets gefunden")


🔍 Extrahiere PropertySets...
✅ 11254 Property-Werte extrahiert
✅ 22 PropertySets gefunden


In [ ]:
# 💾 CSV Export: Detaillierte Daten
if detailed_data:
    df_detailed = pd.DataFrame(detailed_data)
    output_file = 'property_sets_detailed.csv'
    df_detailed.to_csv(output_file, index=False, encoding='utf-8-sig')
    
    print(f"✅ CSV gespeichert: {output_file}")
    print(f"   Zeilen: {len(df_detailed):,}")
    display(df_detailed.head(10))
    
    # Download in Colab
    try:
        from google.colab import files
        files.download(output_file)
        print(f"📥 Download gestartet: {output_file}")
    except:
        pass


✅ CSV gespeichert: property_sets_detailed.csv
   Zeilen: 11,254


,Element_GlobalId,Element_Type,Element_Name,PropertySet,Property,Value
0,11EjrZoVgVGQm8NAB8Ffig,IfcBeam,Beam rafter,Pset_BeamCommon,Reference,Rafters 71 x 171
1,11EjrZoVgVGQm8NAB8Ffig,IfcBeam,Beam rafter,Pset_BeamCommon,FireRating,R 90
2,11EjrZoVgVGQm8NAB8Ffig,IfcBeam,Beam rafter,Pset_BeamCommon,LoadBearing,True
3,11EjrZoVgVGQm8NAB8Ffig,IfcBeam,Beam rafter,Pset_BeamCommon,IsExternal,False
4,11EjrZoVgVGQm8NAB8Ffig,IfcBeam,Beam rafter,Pset_FireRatingProperties,FireResistanceRating,60
5,11EjrZoVgVGQm8NAB8Ffig,IfcBeam,Beam rafter,BaseQuantities,NetSurfaceAreaExtrudedSides,4.46248
6,11EjrZoVgVGQm8NAB8Ffig,IfcBeam,Beam rafter,BaseQuantities,Length,9220.0
7,11EjrZoVgVGQm8NAB8Ffig,IfcBeam,Beam rafter,BaseQuantities,Width,71.0
8,11EjrZoVgVGQm8NAB8Ffig,IfcBeam,Beam rafter,BaseQuantities,Depth,171.0
9,11EjrZoVgVGQm8NAB8Ffig,IfcBeam,Beam rafter,BaseQuantities,CrossSectionArea,0.012141


In [ ]:
# 💾 CSV Export: Zusammenfassung
if summary_data:
    summary_rows = [
        {'PropertySet': pset, 'Property': prop}
        for pset, props in summary_data.items()
        for prop in sorted(props)
    ]
    
    df_summary = pd.DataFrame(summary_rows).sort_values(['PropertySet', 'Property']).reset_index(drop=True)
    output_file = 'property_sets_summary.csv'
    df_summary.to_csv(output_file, index=False, encoding='utf-8-sig')
    
    print(f"✅ CSV gespeichert: {output_file}")
    print(f"   Zeilen: {len(df_summary):,}")
    display(df_summary.head(20))
    
    # Download in Colab
    try:
        from google.colab import files
        files.download(output_file)
        print(f"📥 Download gestartet: {output_file}")
    except:
        pass


✅ CSV gespeichert: property_sets_summary.csv
   Zeilen: 106


,PropertySet,Property
0,BaseQuantities,Area
1,BaseQuantities,CrossSectionArea
2,BaseQuantities,Depth
3,BaseQuantities,GrossArea
4,BaseQuantities,GrossFootprintArea
5,BaseQuantities,GrossSideArea
6,BaseQuantities,GrossSurfaceArea
7,BaseQuantities,GrossVolume
8,BaseQuantities,Height
9,BaseQuantities,Length


In [ ]:
# 📊 Statistiken
print("📊 Top 10 PropertySets:")
pset_counts = df_summary.groupby('PropertySet').size().sort_values(ascending=False)
for pset, count in pset_counts.head(10).items():
    print(f"   {count:3d} Properties - {pset}")

print(f"\n🏗️ Top 10 Element-Klassen:")
element_counts = df_detailed['Element_Class'].value_counts()
for elem_type, count in element_counts.head(10).items():
    print(f"   {count:4d} Properties - {elem_type}")


📊 Top 10 PropertySets:
    20 Properties - BaseQuantities
    14 Properties - Material
    10 Properties - Door Lining Properties
     6 Properties - Pset_DoorCommon
     5 Properties - Pset_CurtainWallCommon
     5 Properties - Pset_DoorWindowGlazingType
     5 Properties - Pset_WindowCommon
     4 Properties - Door Panel Properties - Panel 1
     4 Properties - Door Panel Properties - Panel 2
     4 Properties - Pset_BeamCommon

🏗️ Top 10 Element-Typen:
   2942 Properties - IfcWindow
   2478 Properties - IfcDoor
   2231 Properties - IfcWall
   1601 Properties - IfcCurtainWall
    698 Properties - IfcBeam
    484 Properties - IfcCovering
    252 Properties - IfcColumn
    237 Properties - IfcFurnishingElement
    102 Properties - IfcSlab
     94 Properties - IfcFlowTerminal
